In [2]:
import fragmentAndSimulate as fas
import calcIsotopologues as ci
import pandas as pd
import basicDeltaOperations as op

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def MPADataframe(deltas):
    IDList = ['Ccarbon','Hmethyl','Hother']
    elIDs = ['C','H','H']
    numberAtSite = [1,3,2]

    l = [elIDs, numberAtSite, deltas]
    cols = ['IDS','Number','deltas']

    molecularDataFrame = pd.DataFrame(l, columns = IDList)
    molecularDataFrame = molecularDataFrame.transpose()
    molecularDataFrame.columns = cols

    return molecularDataFrame

In [26]:
def calcComparison(deltaMethylSmp, deltaMethylStd, deltaHydroxylSolvent, deltaHydroxylAtmo):
    '''
    Inputs:
        deltaMethylSmp = delta value of the methyl group in the sample
        deltaMethylStd = delta value of the methyl group in the standard
        deltaHydroxylSolvent = delta value of the equilibrating solvent pool in the Orbitrap measurement
        deltaHydroxylAtmo = delta value of the equilibrating atmospheric pool in the EA measurement. 

    Outputs: The delta values for sample vs standard recorded by the Orbitrap and EA, and the actual sample standard comparison for the methyl group.     
    '''
    #Get isotopologues for the Orbitrap measurement
    MPAStd = MPADataframe([-30,deltaMethylStd, deltaHydroxylSolvent])
    byAtomStdOrbi = ci.inputToAtomDict(MPAStd, disable = True)

    MPASmp = MPADataframe([-30, deltaMethylSmp, deltaHydroxylSolvent])
    byAtomSmpOrbi = ci.inputToAtomDict(MPASmp, disable = True)

    #Calculate the observed Orbitrap delta value
    UValStdPartial = (1/2 * byAtomStdOrbi['000001']['Conc'] + byAtomStdOrbi['000100']['Conc']) / byAtomStdOrbi['000000']['Conc']
    UValSmpPartial = (1/2 * byAtomSmpOrbi['000001']['Conc'] +  byAtomSmpOrbi['000100']['Conc']) / byAtomSmpOrbi['000000']['Conc']
    deltaComparisonOrbi = 1000 * (UValSmpPartial / UValStdPartial - 1)

    #Get isotopologues for the EA measurement
    MPAStd = MPADataframe([-30,deltaMethylStd, deltaHydroxylAtmo])
    byAtomStdEA = ci.inputToAtomDict(MPAStd, disable = True)

    MPASmp = MPADataframe([-30, deltaMethylSmp, deltaHydroxylAtmo])
    byAtomSmpEA = ci.inputToAtomDict(MPASmp, disable = True)

    #Calculate the observed EA delta value
    UValStdEA = (byAtomStdEA['000001']['Conc'] + byAtomStdEA['000100']['Conc']) / byAtomStdEA['000000']['Conc']
    UValSmpEA = (byAtomSmpEA['000001']['Conc'] + byAtomSmpEA['000100']['Conc']) / byAtomSmpEA['000000']['Conc']
    deltaComparisonEA = 1000 * (UValSmpEA / UValStdEA - 1)

    #Compute the actual sample standard comparison for the methyl group 
    UValStdMethyl = (byAtomStdOrbi['000100']['Conc']) / byAtomStdOrbi['000000']['Conc']
    UValSmpMethyl = (byAtomSmpOrbi['000100']['Conc']) / byAtomSmpOrbi['000000']['Conc']
    deltaComparisonMethyl = 1000 * (UValSmpMethyl / UValStdMethyl - 1)

    return deltaComparisonOrbi, deltaComparisonEA, deltaComparisonMethyl

In [27]:
#Given an atmospheric value of -120 per mil, compute the enrichment vs VSMOW of the methyl group based on the EA-IRMS measurement. 
atmoComp = -120
observedC = op.deltaToConcentration('H',-159.6)
atmoC = op.deltaToConcentration('H',atmoComp)

methylC = tuple([(5*observedC[i] - 2*atmoC[i]) / 5 for i in range(len(observedC))])
deltaMetofMPASIG = op.ratioToDelta('H',op.concentrationToM1Ratio(methylC))
deltaMetofMPASIG

-185.9997286420867

In [28]:
#Get scaling constant for EA
comp = calcComparison(-100, deltaMetofMPASIG, -80, -120)
print("EA Scaling constant k")
print(comp[2] / comp[1])
print("Orbitrap Scaling constant k")
print(comp[2] / comp[0])

EA Scaling constant k
1.720720480458795
Orbitrap Scaling constant k
1.376740251148922
